In [1]:
import os

from magic_pdf.data.data_reader_writer import FileBasedDataWriter, FileBasedDataReader
from magic_pdf.data.dataset import PymuDocDataset
from magic_pdf.model.doc_analyze_by_custom_model import doc_analyze
from magic_pdf.config.enums import SupportedPdfParseMethod

/home/jason_sun/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jason_sun/miniconda3/envs/llm/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


import tensorrt_llm failed, if do not use tensorrt, ignore this message


In [2]:
pdf_file_name = "CA899647(A) - DuPont benzaldehyde (1).pdf" 
name_without_suff = pdf_file_name.split(".")[0]

In [3]:
local_image_dir, local_md_dir = "output/images", "output"
image_dir = str(os.path.basename(local_image_dir))
image_writer, md_writer = FileBasedDataWriter(local_image_dir), FileBasedDataWriter(
    local_md_dir
)

In [4]:
# read bytes
reader1 = FileBasedDataReader("file")
pdf_bytes = reader1.read(pdf_file_name)  # read the pdf content

In [5]:
# proc
## Create Dataset Instance
ds = PymuDocDataset(pdf_bytes)

2025-03-10 15:08:09.193 | INFO     | magic_pdf.data.dataset:__init__:156 - lang: None


In [6]:
## inference
if ds.classify() == SupportedPdfParseMethod.OCR:
    infer_result = ds.apply(doc_analyze, ocr=True)

    ## pipeline
    pipe_result = infer_result.pipe_ocr_mode(image_writer)

else:
    infer_result = ds.apply(doc_analyze, ocr=False)

    ## pipeline
    pipe_result = infer_result.pipe_txt_mode(image_writer)

2025-03-10 15:08:18.740 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 12737, cid_chars_radio: 0.0
2025-03-10 15:08:18.741 | WARNING  | magic_pdf.filter.pdf_classify_by_type:classify:335 - OCR needed based on classification result, by_image_area: False, by_text: True, by_avg_words: True, by_img_num: True, by_img_narrow_strips: True, by_invalid_chars: True
2025-03-10 15:08:18.743 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:78 - DocAnalysis init, this may take some times, layout_model: doclayout_yolo, apply_formula: True, apply_ocr: True, apply_table: True, table_model: rapid_table, lang: None
2025-03-10 15:08:18.743 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:92 - using device: cpu
2025-03-10 15:08:18.744 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:96 - using models_dir: /home/jason_sun/.cache/huggingface/hub/models--opendatalab--PDF-Extract-Kit-1.0/snapshots/60416a2cabad3f7b7284b43ce37a99864484fba2/models


CustomVisionEncoderDecoderModel init
VariableUnimerNetModel init
VariableUnimerNetPatchEmbeddings init
VariableUnimerNetModel init
VariableUnimerNetPatchEmbeddings init
CustomMBartForCausalLM init
CustomMBartDecoder init


2025-03-10 15:08:24,094 - DownloadModel - DEBUG: /home/jason_sun/miniconda3/envs/llm/lib/python3.10/site-packages/rapid_table/models/slanet-plus.onnx already exists
2025-03-10 15:08:24.385 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:174 - DocAnalysis init done!
2025-03-10 15:08:24.386 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:custom_model_init:128 - model init cost: 5.643768072128296
2025-03-10 15:08:26.541 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:210 - layout detection time: 2.07
2025-03-10 15:08:28.926 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:216 - mfd time: 2.38
2025-03-10 15:08:28.928 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:223 - formula nums: 0, mfr time: 0.0
2025-03-10 15:08:29.844 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:255 - ocr time: 0.92
2025-03-10 15:08:29.845 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:297 - table time: 0.0
2025-03-10 15:08:29.845 | INFO     | magic_pdf.model.doc_analyz

In [7]:
### draw model result on each page
infer_result.draw_model(os.path.join(local_md_dir, f"{name_without_suff}_model.pdf"))

### get model inference result
model_inference_result = infer_result.get_infer_res()

In [8]:
### draw layout result on each page
pipe_result.draw_layout(os.path.join(local_md_dir, f"{name_without_suff}_layout.pdf"))

### draw spans result on each page
pipe_result.draw_span(os.path.join(local_md_dir, f"{name_without_suff}_spans.pdf"))

In [9]:
### dump markdown
pipe_result.dump_md(md_writer, f"{name_without_suff}.md", image_dir)

### get markdown content
md_content = pipe_result.get_markdown(image_dir)

In [10]:
### dump content list
pipe_result.dump_content_list(md_writer, f"{name_without_suff}_content_list.json", image_dir)

### get content list content
content_list_content = pipe_result.get_content_list(image_dir)

In [11]:
### get middle json
middle_json_content = pipe_result.get_middle_json()

### dump middle json
pipe_result.dump_middle_json(md_writer, f'{name_without_suff}_middle.json')